# Study of $q_0$ 

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
# needed imports

Using the likelihood 
$$\mathcal{L}(y|\mu+b,\vec{\chi}) = \frac{e^{-(\mu+b)}(\mu+b)^y}{y!}\frac{e^{-\frac{(\mu-\mu_0)^2}{2\sigma_\mu^2}}}{\sqrt{2\pi\sigma_\mu^2}}\frac{e^{-\frac{(b-b_0)^2}{2\sigma_b^2}}}{\sqrt{2\pi\sigma_b^2}}$$
Which gives (keeping only "interesting terms")
$$\ell(\mu,b) = -(\mu+b)+y\ln(\mu+b) - \frac{1}{2}\left(\frac{\mu-\mu_0}{\sigma_\mu}\right)^2 - \frac{1}{2} \left(\frac{b-b_0}{\sigma_b}\right)^2$$
Studying the case for $\sigma_s\rightarrow\infty$ the third term above becomes zero. From this we can find the MLEs for two cases.

First, finding $\hat\mu$, which is done by setting $\frac{\partial\ell(\mu,b)}{\partial\mu}=0$, this gives us $\hat\mu = y - b_0$. Furthermore we can find $\hat b$ by setting $\frac{\partial\ell(\hat\mu,b)}{\partial b}=0$, giving $\hat b = b_0$

Then for the second case, the null hypothesis we find $\hat{\hat b}$ by setting $\frac{\partial\ell(0,b)}{\partial b}=0$, this yields $\hat{\hat{b}} =\frac{b_0-\sigma_b^2+\sqrt{(b_0-\sigma_b^2)^2+4y\sigma_b^2}}{2}$ 

Using these values for the likelihood ratio test statistics gives us

$$
q_0 = -2[\ell(s=0,\hat{\hat b}) - \ell(\hat s, \hat b)]
$$
Giving 
$$
q_0 = 2(y\ln\frac{y}{\hat{\hat b}} + \hat{\hat b} - y) + \left(\frac{\hat{\hat b} -b_0}{\sigma_b}\right)^2
$$

In [2]:
def log_likelihood(mu, b, y, b_0, db):
    ell = -(mu+b) + y*np.log(mu+b) -0.5*((b-b_0)/db)**2
    return ell

def test_Stat_null(y, b_hhat, b_0, db):
    q_0 = 2*(y*np.log(y/b_hhat)+b_hhat - y) + ((b_hhat-b_0)/db)**2
    return q_0

Using the asympotics approximation of Brazzale et al we start by defining the likelihood root

$$
r(\mu) = sign(\hat\mu-\mu)\sqrt{2(\ell(\hat\mu, \hat b) - \ell(\mu, \hat{\hat b}))} 
$$
where $\ell(\mu,\hat{\hat b}) =\ell_p(\mu)$ is the profile log likelihood. For our purposes we (should) have that
$$
r(0) = \sqrt{q_0}
$$

In [3]:
def likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db):
    ell_p_hat = log_likelihood(mu_hat, b_hat, y, b_0, db)
    ell_p = log_likelihood(mu, b_hhat, y, b_0, db)
    r = np.sign(mu_hat- mu)*np.sqrt(2*(ell_p_hat-ell_p))
    return r

## **First order approximation**

Furthermore, we have the first order modification of the likelihood root (NB when the dimension of $\mu$ is 1, which is the case here)
\begin{equation}
    r^*(\mu) = r(\mu) + \frac{1}{r(\mu)}\log\left(\frac{q(\mu)}{r(\mu)}\right)
\end{equation}
where
$$
q(\mu) = t(\mu) = \sqrt{j_p(\hat\mu)} (\hat\mu - \mu)
$$
is the Wald statistic, or
$$
q(\mu) = s(\mu) = \sqrt{j_p(\hat\mu)}^{-1}\frac{\partial\ell_p(\mu)}{\partial\mu}
$$
is the score statistic. And where 
$$
j_p(\mu) = -\frac{\partial^2\ell_p(\mu)}{\partial\mu^2}
$$
is the observed information function. 

For our purposes, we have that 
$$
\frac{\partial\ell_p(\mu)}{\partial\mu} = -1 + \frac{y}{\mu + \hat{\hat b}}
$$
and
$$
j_p(\mu) = -\frac{\partial^2\ell_p(\mu)}{\partial\mu^2} = \frac{y}{(\mu + \hat{\hat b})^2}
$$

The first order modification works models that are part the exponential family (or transformation family) making it a good approximation. Which the model we are studying in this notebook (or generally in HEP) is not. Meaning that this is not a good "upgrade" of the asymptotic for our purposes, as we shall soon see

In [4]:
def info_func(mu, b_hatt, y):
    j = y/(mu+b_hatt)**2
    return j

def wald(mu_hat, mu, b_hatt, y, b_hat, db):
    j = info_func(mu_hat, b_hatt, y)
    t = np.sqrt(j)*(mu_hat-mu)*np.sqrt(  np.abs(y/(mu_hat + b_hat)**2 +1/db**2)/np.abs(y/(mu + b_hatt)**2 +1/db**2)   )
    return t

def score(mu_hat, mu, b_hatt, y):
    j = info_func(mu_hat, b_hatt, y)
    s = np.sqrt(1/j)*( -1 + y/(mu+b_hatt))
    return s

def modified_likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db, doWald=True):
    r_mu = likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db)
    if doWald == True:
        q_mu = wald(mu_hat, mu, b_hhat, y, b_hat, db)
    else: 
        q_mu = score(mu_hat, mu, b_hhat, y)
    return r_mu + (1 / r_mu) * np.log(q_mu / r_mu)


Now for an example using $y=3$, $b_0=0.78$ and $\sigma_b=0.18$, first we compare that indeed $r(0)=\sqrt q_0$

In [5]:
y_obs = 3
b_0 = 0.78
sigma_b = 0.18

mu_hat = y_obs - b_0
b_hatt = (b_0-sigma_b**2+np.sqrt((b_0-sigma_b**2)**2+4*y_obs*(sigma_b**2)))/2
b_hat = b_0

r = likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b)

q_0 = test_Stat_null(y_obs, b_hatt, b_0, sigma_b)

print(r)
print(np.sqrt(q_0))

1.8477368150191664
1.8477368150191662


Now we check all the modifications

In [6]:
r_s = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b) # Wald
r_ss = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b, doWald=False) # Score
r_mp = likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs+0.5, b_0, sigma_b) # Mid-p
r_s_mp = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs+0.5, b_0, sigma_b)
r_ss_mp = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs+0.5, b_0, sigma_b, doWald=False)


r_sf = stats.norm.sf(r, loc=0)
r_star_sf = stats.norm.sf(r_s, loc=0)
r_stars_sf = stats.norm.sf(r_ss, loc=0)
r_sf_mp = stats.norm.sf(r_mp, loc=0)
r_star_sf_mp = stats.norm.sf(r_s_mp, loc=0)
r_stars_sf_mp = stats.norm.sf(r_ss_mp, loc=0)



In [7]:
#Making a table of the results...

data_sf = {'True value': [0.0264, 1.94], 'r': [r_sf, r], 'r* Wald': [r_star_sf, r_s], 'r* score': [r_stars_sf, r_ss], 
        'Mid-P r': [r_sf_mp, r_mp], 'Mid-P r* Wald': [r_star_sf_mp, r_s_mp], 'Mid-P r* score': [r_stars_sf_mp, r_ss_mp]}
df= pd.DataFrame(data_sf)
df.index = ['p-value', 'Significance']

df 


,True value,r,r* Wald,r* score,Mid-P r,Mid-P r* Wald,Mid-P r* score
p-value,0.0264,0.032320,0.054249,0.010171,0.015410,0.027993,0.005342
Significance,1.9400,1.847737,1.604981,2.319975,2.159381,1.911152,2.552854


## **Higher order density approximations**

To get better results we need to go to higher order approximations of the asymptotic, which means modifying Eq. (1) wrt to these values for $q$:
$$
q_1 = \frac{|\ell_{;\hat\theta}(\hat\theta) - \ell_{;\hat\theta}(\hat{\hat{\theta}}) \qquad \ell_{b;\hat\theta}(\hat{\hat{\theta}})|}{|\ell_{\theta;\hat\theta}(\hat\theta)|}\left\{\frac{|j_{\theta\theta}(\hat{\theta})|}{|j_{bb}(\hat{\theta}_\psi)|}\right\}^{1/2}
$$
where $|\cdot|$ denotes the determinant, and we are using the notation
$$
\ell_{;X}(Z) = \frac{\partial\ell(\theta)}{\partial X}\Big|_{\theta=Z}\quad\text{and}\qquad \ell_{Y;\ell_\theta(\hat\theta)}(Z) = \frac{\partial\ell(\theta)}{\partial Y\partial \ell_\theta(\hat\theta)^T}\Big|_{\theta=Z}
$$
with $\theta=(\mu,b)$ as a shorthand notation and where $\hat\theta = (\hat\mu,\hat b)$ and $\hat{\hat{\theta}} = (\mu, \hat{\hat b})$



**Alternatively** we could use
$$
q_2 = \frac{|\varphi(\hat\theta) - \varphi(\hat{\hat{\theta}}) \qquad \varphi_{\lambda}(\hat{\hat{\theta}})|}{|\varphi_{\theta}(\hat\theta)|}\left\{\frac{|j_{\theta\theta}(\hat{\theta})|}{|j_{\lambda\lambda}(\hat{\theta}_\psi)|}\right\}^{1/2}
$$
which uses the canonical parameter, $\varphi$, of the tangent exponential model (TEM). Meaning that we have to transform our likelihood. Which is undesirable?


### **Calculations to get the final result...** $q_1$ Edition
Before anything we have to transform the log-likelihood $\ell(\theta,y)$ to be a function of the MLE's and some ancilliary statistics $\ell(\theta;\hat\theta,a)$. We repeat the log-likelhood
$$
\ell(\theta,y) =  -(\mu+b)+y\ln(\mu+b) -\frac{1}{2}\left(\frac{b-b_0}{\sigma_b}\right)^2
$$
We know from the MLEs that we have
$$
\hat\mu = y-b_0,\quad\text{and}\qquad \hat b =b_0
$$
meaning that we can rewrite
$$
y = \hat\mu+\hat b 
$$
***which is what we expect when $y$ is from a Poisson counting experiment***, additionally we can in this example find the simplest ancilliary statistic choice by setting
$$
a=y-(\hat\mu+\hat b) = 0
$$
Thus we can rewrite the log-likelihood to
$$
\ell(\theta;\hat\theta,a) =  -(\mu+b)+(\hat\mu + \hat b +a)\ln(\mu+b) -\frac{1}{2}\left(\frac{b-\hat{b}}{\sigma_b}\right)^2
$$
or since we know $a$
$$
\boxed{\ell(\theta;\hat\theta,a) =  -(\mu+b)+(\hat\mu + \hat b)\ln(\mu+b) -\frac{1}{2}\left(\frac{b-\hat{b}}{\sigma_b}\right)^2}
$$

#### **Calculating $\ell_{;\hat\theta}(\hat\theta)$**
Starting the calculations... we have first that
$$
\ell_{;\hat\theta}(\hat\theta) = \begin{pmatrix} \frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0}\\
\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=y-b_0,b=b_0}\end{pmatrix}
$$
where 
$$
\frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0} = \frac{\partial}{\partial\hat\mu}\left( -(\mu+b)+(\hat\mu+\hat{b})\ln(\mu+b) -\frac{1}{2}\left(\frac{b-\hat{b}}{\sigma_b}\right)^2\right)\Big|_{\mu=y-b_0,b=b_0} = \log(\mu+b)\Big|_{\mu=y-b_0,b=b_0} = \log y
$$
Similarly for $b$ we have
$$
\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=y-b_0,b=b_0} = \frac{\partial}{\partial\hat\mu}\left( -(\mu+b)+(\hat\mu+\hat b)\ln(\mu+b) -\frac{1}{2}\left(\frac{b-\hat b}{\sigma_b}\right)^2\right)\Big|_{\mu=y-b_0,b=b_0} = \log(\mu+b) + \frac{b-\hat b}{\sigma_b^2}\Big|_{\mu=y-b_0,b=b_0} = \log(y)
$$
Giving 
$$
\boxed{\ell_{;\hat\theta}(\hat\theta) = \begin{pmatrix} \log y\\ \log y\end{pmatrix}}
$$



#### **Calculating $\ell_{;\hat\theta}(\hat{\hat{\theta}})$**
Next we have that
$$
\ell_{;\hat\theta}(\hat{\hat{\theta}}) = \begin{pmatrix} \frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=\mu,b=\hat{\hat b}}\\
\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=\mu,b=\hat{\hat b}}\end{pmatrix}
$$
using the previous results we get 
$$
\boxed{\ell_{;\hat\theta}(\hat{\hat{\theta}}) = \begin{pmatrix} \log(\mu+\hat{\hat b})\\  \log(\mu+\hat{\hat b}) +\frac{\hat{\hat b}-\hat{b}}{\sigma_b^2}\end{pmatrix}}
$$



#### **Calculating $\ell_{b;\hat\theta}(\hat{\hat{\theta}})$**
Next we have that
$$
\ell_{b;\hat\theta}(\hat{\hat{\theta}}) = \begin{pmatrix} \frac{\partial^2\ell(\mu,b)}{\partial b \partial\hat\mu^T}\Big|_{\mu=\mu,b=\hat{\hat b}}\\
\frac{\partial^2\ell(\mu,b)}{\partial b\partial\hat b}\Big|_{\mu=\mu,b=\hat{\hat b}}\end{pmatrix}
$$
Using the previous results, we have that 
$$
\frac{\partial}{\partial b}\frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{\partial}{\partial b}\log(\mu+b)\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{1}{\mu + b}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{1}{\mu + \hat{\hat b}}
$$
and
$$
\frac{\partial}{\partial b}\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{\partial}{\partial b}\log(\mu+b) +\frac{b-\hat b}{\sigma_b^2}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{1}{\mu + \hat{\hat b}} + \frac{1}{\sigma_b^2}
$$
giving
$$
\boxed{\ell_{b;\hat\theta}(\hat{\hat{\theta}}) = \begin{pmatrix} \frac{1}{\mu + \hat{\hat b}}\\  \frac{1}{\mu + \hat{\hat b}} + \frac{1}{\sigma_b^2}\end{pmatrix}}
$$

#### **Calculating $\ell_{\theta;\hat\theta}(\hat\theta)$**
Starting the calculations... we have first that
$$
\ell_{\theta;\hat\theta}(\hat\theta) = \begin{pmatrix} \frac{\partial^2\ell(\mu,b)}{\partial \mu \partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0} &
\frac{\partial^2\ell(\mu,b)}{\partial b \partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0}\\
\frac{\partial^2\ell(\mu,b)}{\partial\mu\partial\hat b}\Big|_{\mu=y-b_0,b=b_0} &
\frac{\partial^2\ell(\mu,b)}{\partial b\partial\hat b}\Big|_{\mu=y-b_0,b=b_0}\end{pmatrix}
$$
We almost have all of these terms! Starting by finding the easiest term
$$
\frac{\partial^2\ell(\mu,b)}{\partial \mu \partial\hat b}\Big|_{\mu=y-b_0,b=b_0} = \frac{1}{\mu+b}\Big|_{\mu=y-b_0,b=b_0} = \frac{1}{y}
$$
Next we gotta find
$$
\frac{\partial^2\ell(\mu,b)}{\partial \mu \partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0} = \frac{1}{\mu+b}\Big|_{\mu=y-b_0,b=b_0} = \frac{1}{y}
$$
Such that we have
$$
\boxed{\ell_{\theta;\hat\theta}(\hat\theta) = \begin{pmatrix} \frac{1}{y} &
\frac{1}{y}\\
\frac{1}{y} &
\frac{1}{y} + \frac{1}{\sigma_b^2}\end{pmatrix} }
$$

#### **Jacobian temrs**
Lastly we have the Jacobian terms. Where we can identify that
$$
j_{\theta\theta}(\hat\theta) = -\ell_{\theta;\hat\theta}(\hat\theta) 
$$
giving
$$
\Rightarrow |j_{\theta\theta}(\hat\theta)| = \frac{1}{y\sigma_b^2}
$$

The other term we already have, it just has to be evaluated for other values:
$$
j_{bb}(\hat{\hat{\theta}}) =  -\frac{\partial^2\ell(\mu,b)}{\partial b\partial b^T}\Big|_{\mu=\mu,b=\hat{\hat b}}  = \frac{y}{(\mu+\hat{\hat b})^2} + \frac{1}{\sigma_b^2}
$$

#### **Calculating the determinants**
Lastly comes the tedious part of calculating the determinant of the matrices we will get. First we have
$$
|\ell_{;\hat\theta}(\hat\theta) - \ell_{;\hat\theta}(\hat{\hat{\theta}}) \qquad \ell_{b;\hat\theta}(\hat{\hat{\theta}})| = 
\left|\begin{vmatrix}\log\left(\frac{y}{\mu+\hat{\hat b}}\right) &
\frac{1}{\mu + \hat{\hat b}}\\  
\log\left(\frac{y}{\mu+\hat{\hat b}}\right) - \frac{\hat{\hat b}-\hat{b}}{\sigma_b^2}&
\frac{1}{\mu + \hat{\hat b}} + \frac{1}{\sigma_b^2} \end{vmatrix}\right| = \left| \frac{\log\left(\frac{y}{\mu+\hat{\hat b}}\right)}{\sigma_b^2} - \frac{(\hat{\hat b}-\hat{b})}{(\mu + \hat{\hat b})\sigma_b^2} \right|
$$
and
$$
|\ell_{\theta;\hat\theta}(\hat\theta)| = \frac{1}{y\sigma_b^2}
$$


#### Putting it all together
We now have a formula for the higher order approximation
$$
q_1 = \left| y\log\left(\frac{y}{\mu+\hat{\hat b}}\right) - y\frac{(\hat{\hat b}-\hat{b})}{\mu + \hat{\hat b}} \right|
\left[\frac{\frac{1}{y\sigma_b^2}}{\frac{y}{(\mu+\hat{\hat b})^2} + \frac{1}{\sigma_b^2}}\right]^{1/2}
$$
So now we can test this result!

In [8]:
def higher_order_terms(mu, b_hat, b_hhat, y, db):
        q = np.abs(np.log(y/(mu+b_hhat))*y -y*(b_hhat-b_hat)/((mu+b_hhat)))
        jacobian = np.sqrt(  np.abs(1/(y*db**2)) / np.abs(1/db**2 + (y/(mu+b_hhat)**2)))
        return q*jacobian

def modified_likelihood_root_HO(mu, mu_hat, b_hat, b_hhat, y, b_0, db):
        r_mu = likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db)
        q_1 = higher_order_terms(mu, b_hat, b_hhat, y, db)
        return r_mu + (1 / r_mu) * np.log(q_1 / r_mu)

In [9]:
r_s_HO = modified_likelihood_root_HO(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b)

r_sf_HO = stats.norm.sf(r_s_HO, loc=0)

#Making a table of the results...

data_sf = {'True value': [0.0264, 1.94], 'r': [r_sf, r], 'r*': [r_star_sf, r_s], 'r* H.O.': [r_sf_HO, r_s_HO]}
df= pd.DataFrame(data_sf)
df.index = ['p-value', 'Significance']

df 

,True value,r,r*,r* H.O.
p-value,0.0264,0.032320,0.054249,0.031624
Significance,1.9400,1.847737,1.604981,1.857450


# **Skovgaard's approximation**
As the previous result would require the user to transform the log-likelihood, it is not so practical for physicists to use on the fly. Thus we can use an alternative method of finding the Higher order approximation by, Skovgaard approximation: 
$$ 
q_1=[\hat{S}^{-1}\hat{Q}]_1|j(\hat{\theta})|^{1/2}|i^{-1}(\hat{\theta})||\hat{S}||j_{bb}(\hat{\theta}_\psi)|^{-1/2} 
$$ 
Where 
$$\hat{S} = S(\hat{\theta},\hat{\theta}_\psi) = cov_{\hat\theta}\{\ell_\theta(\hat\theta),\ell_\theta(\hat{\hat{\theta}})\}$$ 
and 
$$\hat{Q} = Q(\hat{\theta},\hat{\theta}_\psi)= cov_{\hat\theta}\{\ell_\theta(\hat\theta),\ell(\hat\theta) - \ell(\hat{\hat{\theta}})\}$$ 

All of these variables are easily accessible using standard statistical software. A caveat of using Skovgaard's approximation is that it requires generating more samples $y$ and $b_0$ around $\hat\theta$. Below we showcase one way of implementing this numerically using JAX. 

In [10]:
## Needed packages
import jax.numpy as jnp
import jax
from jax import grad, hessian
from scipy.optimize import minimize
# np.random.seed(42)  # Ensure reproducibility

First we find the MLEs numerically, just to show that is possible to generalize this completely for any log-likelihood


In [11]:
# Find MLE using scipy
# Define the log-likelihood function using JAX-compatible functions
def log_likelihood(theta, y, lamda0, sigma_lamda):
    psi, lamda = theta
    return -(psi + lamda) + y * jnp.log(psi + lamda) - 0.5 * ((lamda - lamda0) / sigma_lamda) ** 2

# Define the log-likelihood function for MLE search
def log_likelihood_np(theta, y, lamda0, sigma_lamda):
    psi, lamda = theta
    return -(psi + lamda) + y * np.log(psi + lamda) - 0.5 * ((lamda - lamda0) / sigma_lamda) ** 2

# Define the profile log-likelihood function for MLE search
def profile_likelihood(lamda, psi_fixed, y, lamda0, sigma_lamda):
    return -log_likelihood_np([psi_fixed, lamda], y, lamda0, sigma_lamda)

mu = 0 # background hypothesis
# Find MLE using scipy
initial_guess = [1.0, 1.0]
mle_result = minimize(lambda theta: -log_likelihood_np(theta, y_obs, b_0, sigma_b), initial_guess)
hat_theta = mle_result.x

mle_profile_result = minimize(lambda lamda: profile_likelihood(lamda, mu, y_obs, b_0, sigma_b), [hat_theta[1]])
hat_theta_psi = jnp.array([mu, mle_profile_result.x[0]])

print(f'Estimated MLEs: {hat_theta} \nAnalytical MLEs: {[mu_hat, b_hat]}')
print(f'Estimated profile MLEs: {hat_theta_psi} \nAnalytical profile MLEs: {[mu, b_hatt]}')

Estimated MLEs: [2.22000001 0.77999999] 
Analytical MLEs: [2.2199999999999998, 0.78]
Estimated profile MLEs: [0.        0.8605509] 
Analytical profile MLEs: [0, np.float64(0.8605509013859143)]


Now we initialize all of the JAX functions we need to vectorize this

In [12]:
# Use automatic differentiation for gradients and Hessians
gradient = grad(log_likelihood, argnums=0)  # Gradient w.r.t theta
hessian_func = hessian(log_likelihood, argnums=0)  # Hessian w.r.t theta

grad_vmap = jax.vmap(lambda y, b_0: gradient(hat_theta, y, b_0, sigma_b))     
grad_vmap_psi = jax.vmap(lambda y, b_0: gradient(hat_theta_psi, y, b_0, sigma_b))

ll_vmap = jax.vmap(lambda y, b_0: log_likelihood(hat_theta, y, b_0, sigma_b))
ll_vmap_psi = jax.vmap(lambda y, b_0: log_likelihood(hat_theta_psi, y, b_0, sigma_b))

info_vmap = jax.vmap(lambda y, b_0: hessian_func(hat_theta, y, b_0, sigma_b))

Here we have created four functions of the samples $y$ and $b_0$
$$
\ell_\theta(\hat\theta, y, b_0),\quad\ell_\theta(\hat{\hat{\theta}}, y, b_0),\quad\ell(\hat\theta, y,b_0),\quad\text{and}\quad\ell(\hat{\hat{\theta}}, y,b_0),
$$
respectively for all "vmap"s

Next we compute $j(\hat\theta)$ and $j_{bb}(\hat{\hat{\theta}})$, which is easily computed as: 

In [13]:
# Compute fisher info using JAX
j_hat = -hessian_func(hat_theta, y_obs, b_0, sigma_b)
j_bb = -hessian_func(hat_theta_psi, y_obs, b_0, sigma_b)[-1,-1]

print(j_hat)
print(j_bb)

[[ 0.33333334  0.33333334]
 [ 0.33333334 31.197529  ]]
34.91525


Lastly we need to generate more samples to compute the expected fisher information $i(\hat\theta)=E[j(\hat\theta)]$, $\hat Q$, and $\hat S$. To do this we generate samples that are centered around $\hat\theta$. This we do by computing new NPs with $b_0 \sim\mathcal{N}(\hat{b}, \sigma_b^2)$ and new observations from $y\sim\text{Poisson}(\hat\mu + \hat b)$. For this example we generate 10000 samples

In [14]:
n_boots = 100000
b_bootstrap = np.random.normal(loc=hat_theta[1], scale=sigma_b, size=n_boots)
y_bootstrap = np.random.poisson(lam=hat_theta[0] + hat_theta[1], size=n_boots)

Next we calculate $\hat S$. This we do by calculating $\ell_\theta(\hat\theta, y)$ and $\ell_\theta(\hat{\hat{\theta}}, y)$ for every generated "y_bootstrap" and "b_bootstrap" sample we generated. Then we calculate the covariance between the two scores, but we remember that we have

$$
\text{cov}(\ell_\theta(\hat\theta), \ell_\theta(\hat{\hat{\theta}})) 
= 
\begin{pmatrix}
    \text{Cov}(\ell_\theta(\hat\theta), \ell_\theta(\hat\theta)) & \text{Cov}(\ell_\theta(\hat\theta), \ell_\theta(\hat{\hat{\theta}})) \\
    \text{Cov}(\ell_\theta(\hat{\hat{\theta}}), \ell_\theta(\hat\theta)) & \text{Cov}(\ell_\theta(\hat{\hat{\theta}}), \ell_\theta(\hat{\hat{\theta}}))
\end{pmatrix}
$$
$$
= 
\begin{pmatrix}
    \text{Var}(\ell_\mu(\hat\mu)) & \text{Cov}(\ell_\mu(\hat\mu), \ell_b(\hat b)) & \text{Cov}(\ell_\mu(\hat\mu), \ell_\mu(\mu)) &\text{Cov}(\ell_\mu(\hat\mu), \ell_b(\hat{\hat b}) \\
    \text{Cov}(\ell_b(\hat b), \ell_\mu(\hat\mu)) & \text{Var}(\ell_b(\hat b)) & \text{Cov}(\ell_b(\hat b), \ell_\mu(\mu)) &\text{Cov}(\ell_b(\hat b), \ell_b(\hat{\hat b}) \\
    \text{Cov}(\ell_\mu(\mu), \ell_\mu(\hat\mu)) & \text{Cov}(\ell_\mu(\mu), \ell_b(\hat b)) & \text{Var}(\ell_\mu(\mu)) &\text{Cov}(\ell_\mu(\mu), \ell_b(\hat{\hat b}) \\
    \text{Cov}(\ell_b(\hat{\hat b}), \ell_\mu(\hat\mu)) & \text{Cov}(\ell_b(\hat{\hat b}), \ell_b(\hat b)) & \text{Cov}(\ell_b(\hat{\hat b}), \ell_\mu(\mu)) &\text{Var}(\ell_b(\hat{\hat b})
\end{pmatrix}
$$
So we are interested in the upper right four elements. To get $\hat S$ we use the following script

In [15]:
score_vectors = grad_vmap(y_bootstrap, b_bootstrap)
score_vectors_psi = grad_vmap_psi(y_bootstrap, b_bootstrap)

# showing the full covariance matrix
S_hat = np.cov(score_vectors.T, score_vectors_psi.T)
print(S_hat)

# the part that is of interest to us
S_hat = S_hat[:len(hat_theta), len(hat_theta):]
print(S_hat)

S_hat += 1e-8 * np.eye(S_hat.shape[0])  # Regularization for numerical stability

[[ 0.33322312  0.32118786  1.16166214  1.14962687]
 [ 0.32118786 30.99279018  1.11970554 31.7913077 ]
 [ 1.16166214  1.11970554  4.04971578  4.00775914]
 [ 1.14962687 31.7913077   4.00775914 34.64943977]]
[[ 1.16166214  1.14962687]
 [ 1.11970554 31.7913077 ]]


Next we calculate $\hat Q$, which follows the same procedure as described above,
$$
\text{cov}(\ell_\theta(\hat\theta), \Delta\ell) 
= 
\begin{pmatrix}
    \text{Var}(\ell_\mu(\hat\mu)) & \text{Cov}(\ell_\mu(\hat\mu), \ell_b(\hat b)) & \text{Cov}(\ell_\mu(\hat\mu), \Delta\ell) \\
    \text{Cov}(\ell_b(\hat b), \ell_\mu(\hat\mu)) & \text{Var}(\ell_b(\hat b)) & \text{Cov}(\ell_b(\hat b), \Delta\ell) \\
    \text{Cov}(\Delta\ell, \ell_\mu(\hat\mu)) & \text{Cov}(\Delta\ell, \ell_b(\hat b)) & \text{Var}(\Delta\ell)
\end{pmatrix}
$$
where to spare notation I have written: $\Delta\ell = \ell(\hat\theta)-\ell(\hat{\hat{\theta}})$


We compute this below

In [16]:
ll_bootstrap_mle = ll_vmap(y_bootstrap, b_bootstrap)
ll_bootstrap_mle_psi = ll_vmap_psi(y_bootstrap, b_bootstrap)

ll_diffs = ll_bootstrap_mle - ll_bootstrap_mle_psi

Q_hat = np.cov(score_vectors_psi.T, ll_diffs)  # Covariance between score vector and scalar of differences of log-likelihoods

print(Q_hat)

Q_hat = Q_hat[:-1, -1]  # Covariance between score vector and scalar

print(Q_hat)

[[ 4.04971578  4.00775914  4.35541282]
 [ 4.00775914 34.64943977  1.83872917]
 [ 4.35541282  1.83872917  4.88327212]]
[4.35541282 1.83872917]


Next we find the expected Fisher information, and inverse $\hat S$

In [17]:
j_hats = -info_vmap(y_bootstrap, b_bootstrap)
i_hat = np.mean(j_hats, axis=0)

S_inv =jnp.linalg.inv(S_hat)
print(i_hat)

print(S_inv)

[[ 0.33371446  0.33371446]
 [ 0.33371446 31.197906  ]]
[[ 0.891924   -0.03225346]
 [-0.03141401  0.03259112]]


With all of this we now can compute $q_1$

In [18]:
q_1 = np.abs(S_inv @ Q_hat)[0] * jnp.sqrt(jnp.abs(jnp.linalg.det(j_hat))) \
       * jnp.abs(jnp.linalg.det(jnp.linalg.inv(i_hat))) * jnp.abs(jnp.linalg.det(S_hat)) / jnp.sqrt(jnp.abs(j_bb))

print(q_1)

7.185977


Then with this we get the modified likelihood-root. The results are below

In [19]:
r_skov = r + 1/r*np.log(q_1/r)

r_sf_skov = stats.norm.sf(r_skov, loc=0)

#Making a table of the results...

data_sf = {'True value': [0.0264, 1.94], 'r': [r_sf, r], 'r*': [r_star_sf, r_s], 'r* H.O.': [r_sf_HO, r_s_HO], 'r* Skovgaard': [r_sf_skov, r_skov]}
df= pd.DataFrame(data_sf)
df.index = ['p-value', 'Significance']

df 

,True value,r,r*,r* H.O.,r* Skovgaard
p-value,0.0264,0.032320,0.054249,0.031624,0.004900
Significance,1.9400,1.847737,1.604981,1.857450,2.582782


# **Testing the canonical parameter**
As briefly mentioned, there is an alternative way of calculating this, we have
$$
q_2 = \frac{|\varphi(\hat\theta) - \varphi(\hat{\hat{\theta}}) \qquad \varphi_{\lambda}(\hat{\hat{\theta}})|}{|\varphi_{\theta}(\hat\theta)|}\left\{\frac{|j_{\theta\theta}(\hat{\theta})|}{|j_{\lambda\lambda}(\hat{\theta}_\psi)|}\right\}^{1/2}
$$
where we use the canonical parameter $\varphi$ instead. So now we try to find it!
To recapitulate a bit, we have the transformed log-likelihood
\begin{equation}
\ell(\theta,y) =  -(\mu+b)+y\ln(\mu+b) -\frac{1}{2}\left(\frac{b-b_0}{\sigma_b}\right)^2
\end{equation}
For higher-order asymptotics, one often rewrites the log-likelihood as
$$
\ell(\theta;\hat\theta,a) =  \varphi(\theta)^Ts(y)-h(\theta)+c(y)
$$
where $\varphi(\theta)$ is the **canonical parameter** that we are interested in, $s(y)$ is a *sufficient statistic*, $h(\theta)$ captures additional terms dependent on $\theta$, and $c(y)$ is a term independent of $\theta$, that usually is ignored in inference. 

To construct the canonical parameter we can make use of the general formula
$$
\varphi(\theta) = ^T = \sum_{i=1}^{n}\frac{\partial{\ell(\theta,y)}}{\partial y_i}\Big|_{y=y^0}V_i
$$
where we have that $n=1$ and $y^0=y$, and this $V_i$ is a vector that implement conditioning on an *approximately ancillary statistic*. This $V_i$ is generally constructed from pivotal quantities $z_i(y_i,\theta)$, which always exists in the form of the probability integral transformation $F(y_i:\theta)$, and can be found using the formula
$$
V = -\left(\frac{\partial z}{\partial y^T}\right)^{-1}\left(\frac{\partial z}{\partial \theta^T}\right)\Bigg|_{(y^0,\hat\theta_0)}
$$
So the name of the game is finding $z(y,\theta)$ 


## One choice of the Pivot

**(GPT recommendation following)** a natural choice since $y\sim\text{Poisson}(\theta)$ is to use
$$
z(y,\theta) = \frac{y-(\mu+b)}{\sqrt{\mu +b}}
$$
**WHICH APPLIES THE CLT FOR LARGE $\mu+b$!!!** giving
$$
\left(\frac{\partial z}{\partial y}\right)^{-1} = \sqrt{\mu +b }
$$
and 
$$
\left(\frac{\partial z}{\partial \theta}\right) = \frac{-y+(\mu+b)}{2(\mu+b)^{3/2}}\begin{pmatrix}1\\1\end{pmatrix}
$$
This was not pursued further, as this yields that $q_2=0$, meaning that no HOA is needed. 


## Another choice of the Pivot, from Anthony Davison:
If we interpret this model as having an observation $b_0$ with an unknown mean $b$ and variance $\sigma_b^2$ and a Poisson variable with mean $\mu+b$, then one pivot is
$$
z_1=\frac{b_0-b}{\sigma_b^2}
$$
OR
$$
z_1=\frac{b_0-b}{s_0}
$$

# **Not pursued further !!!!**


$$
\varphi(\mu,b) = \begin{pmatrix}\ln(\mu+b)\\\ln(\mu+b)\end{pmatrix}
$$ is a good canonical parameter, since $s(y)=y=\hat\mu+\hat b+ a$ follows this form.

$\varphi(\theta)^T = \sum_{i=1}^{n}\frac{\partial{ell(\theta,y)}}{\partial y_i}\Big|_{y=y^0}V_i$

### Finding remaining parts
From this it is trivial to find that
$$
\varphi_\theta(\hat{\theta}) = \begin{pmatrix} \frac{\partial\varphi}{\partial\mu}\Big|_{\theta=\hat\theta}\\
\frac{\partial\varphi}{\partial b}\Big|_{\theta=\hat\theta}\end{pmatrix}
= \begin{pmatrix} \frac{1}{\hat\mu+\hat b}&&\frac{1}{\hat\mu+\hat b} \\ \frac{1}{\hat\mu+\hat b}&&\frac{1}{\hat\mu+\hat b}\end{pmatrix}
= \begin{pmatrix} \frac{1}{y}&&\frac{1}{y}\\\frac{1}{y}&&\frac{1}{y}\end{pmatrix}
$$
and even easier now to see that
$$
\varphi_\lambda(\hat{\hat{\theta}}) = \begin{pmatrix}\frac{1}{\mu+\hat{\hat b}}\\\frac{1}{\mu+\hat{\hat b}}\end{pmatrix}
$$

### Putting it all togheter
That means that we now have
$$
|\varphi(\hat\theta) - \varphi(\hat{\hat{\theta}}) \qquad \varphi_{\lambda}(\hat{\hat{\theta}})| = \begin{vmatrix}\log\left(\frac{y}{\mu+\hat{\hat b}}\right)& \frac{1}{\mu+\hat{\hat b}}\\\log\left(\frac{y}{\mu+\hat{\hat b}}\right)& \frac{1}{\mu+\hat{\hat b}}\end{vmatrix}
$$
This yields
$$
|\varphi(\hat\theta) - \varphi(\hat{\hat{\theta}}) \qquad \varphi_{\lambda}(\hat{\hat{\theta}})| = \sqrt{\log\left(\frac{y}{\mu+\hat{\hat b}}\right)^2 + \frac{1}{(\mu+\hat{\hat b})^2}}
$$
Similarly we have
$$
\varphi_\theta(\hat\theta) = \frac{\sqrt{2}}{y^2}
$$
The fisher information term remains the same as for $q_1$ meaning that we have
$$
q_2 = \frac{y^2\sqrt{\log\left(\frac{y}{\mu+\hat{\hat b}}\right)^2 + \frac{1}{(\mu+\hat{\hat b})^2}}}{\sqrt{2}}\left[\frac{\frac{1}{y\sigma_b^2}}{\frac{y}{(\mu+\hat{\hat b})^2} + \frac{1}{\sigma_b^2}}\right]^{1/2}
$$

In [20]:
def higher_order_terms_2(mu, b_hhat, y, db):
        numerator = y**2*np.sqrt(np.log(y/(mu+b_hhat))**2 + 1/(mu+b_hhat)**2 )
        denominator = np.sqrt(2)
        q = numerator/denominator
        jacobian = np.sqrt(  np.abs(1/(y*db**2)) / np.abs(1/db**2 + (y/(mu+b_hhat)**2)))
        return q*jacobian

def modified_likelihood_root_HO_2(mu, mu_hat, b_hat, b_hhat, y, b_0, db):
        r_mu = likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db)
        q_2 = higher_order_terms_2(mu, b_hhat, y, db)
        return r_mu + (1 / r_mu) * np.log(q_2 / r_mu)

In [21]:
r_s_HO2 = modified_likelihood_root_HO_2(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b)

r_sf_HO2 = stats.norm.sf(r_s_HO, loc=0)

#Making a table of the results...

data_sf = {'True value': [0.0264, 1.94],'r': [r_sf, r], 'r*': [r_star_sf, r_s], 'r* H.O.': [r_sf_HO, r_s_HO], 'r* H.O. canon': [r_sf_HO2, r_s_HO2]}
df= pd.DataFrame(data_sf)
df.index = ['p-value', 'Significance']

df 

TypeError: log_likelihood() takes 4 positional arguments but 5 were given

From this we see that using $q_1$ yields a somewhat higher significance than just